In [1]:
import os

import requests
import json
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
index_name = "conversations"
index_path = "../indexes/conversations"
embeddings = OpenAIEmbeddings()
# db = FAISS.from_texts(texts=['9'], embedding=embeddings)
# db.save_local(folder_path=index_path, index_name=index_name) 
# db = FAISS.load_local(folder_path=index_path, index_name=index_name, embeddings=embeddings)


/var/folders/g5/sts_zwf50495g151b7vd71840000gp/T/ipykernel_53154/3321667123.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [4]:
def gpt_system_user(
    system_message: str, user_message: str, model: str = "gpt-3.5-turbo"
):
    """Para usar en notebooks"""
    # Hyper 2
    # POST https://yoizenia.openai.azure.com/openai/deployments/GPT35Turbo/chat/completions

    try:
        response = requests.post(
            url="https://api.openai.com/v1/chat/completions",
            params={
                "temperature": "0",
            },
            headers={
                "Content-Type": "application/json",
                "Authorization": f"Bearer {OPENAI_API_KEY}",
            },
            data=json.dumps(
                {
                    "model": model,
                    "messages": [
                        {"content": system_message, "role": "system"},
                        {"content": user_message, "role": "user"},
                    ]
                }
            ),
            timeout=10,
        )
        choices = response.json()["choices"]
        answer = {
            "status_code": response.status_code,
            "text": choices[0]["message"],
        }
        return answer

    except requests.exceptions.RequestException:
        print("HTTP Request failed")
        return None


In [5]:
condensed_contexts =  [
        {
          "messages": [
              {
                "role": "user",
                "content": "Soy Alexander, mi madre se llama Sylvia, te paso los datos de ella mail: sylvia@gmail.com celu 1123343434."
              },
              {
                "role": "assistant",
                "content": "Excelente Alexander, como puedo ayudarte?"  
              },
              {
                "role": "user",
                "content": "quiero sacar un turno para ella porque no entiende nada de tecnologia y los turnos se sacan por esta via"
              },
              {
                "role": "assistant",
                "content": "No hay problema, sacamos el turno para ella, cual es el DNI de ella?"
              },
              {
                "role": "user",
                "content": "12668992"
              },
              {
                  "role": "assistant",
                  "content": "perfecto, digame que especialidad?"  
              },
              {
                  "role": "user",
                  "content": "ginecologia"
              },
              {
                  "role": "assistant",
                  "content": "Tengo turnos disponibles con el Dr. Facundo Farias para el 27 de agosto a las 10:00hs, le parece bien?"
              },
              {
                "role": "user",
                "content": "si"
              },
              {
                "role": "assistant",
                "content": "confirmado entonces el turno para Sylvia para el 27 de agosto a las 10:00hs con el Dr. Facundo Farias. Le envio el comprobante por mail?"
              },
              {
                "role": "user",
                "content": "si"
              },
              {
                "role": "assistant",
                "content": "Al correo de Sylvia?"
              },
              {
                "role": "user",
                "content": "si"
              },
              {
                "role": "assistant",
                "content": "Perfecto, le envio el comprobante al mail de ella. Algo mas que pueda ayudarte?"
              },
              {
                "role": "user",
                "content": "no, muchas gracias"
              }
           ],
          "metadata": {
            "session_id": "1",
            "person_id": "20",
            "context_id": "1",
            "created_at": "20200825T12:00:00.000Z",
          },
        },     
  ]


In [5]:
db = FAISS.from_texts(texts=['notebook-14'], embedding=embeddings)
for context in condensed_contexts:
        # Join all messages in one string by a comma
        messages = ''.join([str(message) for message in context['messages']])
        # print(messages)
        # print(context['metadata'])
        extraction = gpt_system_user(
            system_message="""
              Extract every meaningful information about the user from the provided conversation.
              Ask yourself, what happened in the conversation?
              What data about the user did you learn?
              At what date and time was this data gathered?

              Create a summary of everything you learned about the user.

              Use this format:

              On [date] at [time] the [user's / user's related person] [data  learned about the user or user's related person].
              """,
            user_message=f" METADATA: ```{context['metadata']} CONVERSATION: ```{messages}``` ```",
            model="gpt-3.5-turbo",
        )
        if extraction['status_code'] != 200:
            print(f"Error: Openai API returned status code {extraction['status_code']}")
            continue
        else:
            print(f"{extraction['text']['content']} \n\n\n///////////////////////////\n\n\n")
            db.add_texts(
                texts=[extraction['text']['content']], 
                metadatas=[context['metadata']],
            )

On August 25, at 12:00 PM the user provided the following information about themselves and their related person:
- User's name: Alexander
- User's related person: Sylvia (mother)
- Sylvia's email: sylvia@gmail.com
- Sylvia's phone number: 1123343434
- Sylvia's DNI (identification number): 12668992

Additionally, the user requested assistance in scheduling a medical appointment for Sylvia in the field of gynecology. The assistant provided available time slots for the appointment and confirmed a specific date and time with the user (August 27, at 10:00 AM with Dr. Facundo Farias). The user confirmed the appointment and requested that the appointment confirmation be sent to Sylvia's email.

In conclusion, the user's name is Alexander and they requested assistance in scheduling a gynecology appointment for their mother, Sylvia. The appointment was successfully scheduled for August 27, at 10:00 AM with Dr. Facundo Farias, and the appointment confirmation will be sent to Sylvia's email. 


/

ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).

Mezclando data y fechas de dos contextos

In [8]:
user_input = "yo tengo algun turno asignado?"
filter = {"person_id": "20"}
db_results = db.similarity_search_with_score(query=user_input, embeddings=embeddings, filter=filter, k=2)
context_k2 = [f'\n {db_results[i][0].page_content} {db_results[i][0].metadata}\n---------------\n\n' for i in range(len(db_results))]
context = context_k2
for c in context:
    print(c)
system_prompt = f"""
You are the personal assistant of the user.
Use the user's name in your answers.
Direct your answers to the user in the second person.
You will find past conversations between you and the user between backticks.
Use them to answer the user's question.
To reason about dates, have in mind that today is September 13th, 2023.
Wait before answering, check when the data was gathered first.
Take a deep breath and work on this problem step-by-step.



```{context}```

"""
# answer = gpt_system_user(system_message=system_prompt, user_message=user_input, model="gpt-4")
answer = gpt_system_user(system_message=system_prompt, user_message=user_input)
# answer["text"]["content"]



 On August 25, at 12:00 PM the user provided the following information about themselves and their related person:
- User's name: Alexander
- User's related person: Sylvia (mother)
- Sylvia's email: sylvia@gmail.com
- Sylvia's phone number: 1123343434
- Sylvia's DNI (identification number): 12668992

Additionally, the user requested assistance in scheduling a medical appointment for Sylvia in the field of gynecology. The assistant provided available time slots for the appointment and confirmed a specific date and time with the user (August 27, at 10:00 AM with Dr. Facundo Farias). The user confirmed the appointment and requested that the appointment confirmation be sent to Sylvia's email.

In conclusion, the user's name is Alexander and they requested assistance in scheduling a gynecology appointment for their mother, Sylvia. The appointment was successfully scheduled for August 27, at 10:00 AM with Dr. Facundo Farias, and the appointment confirmation will be sent to Sylvia's email. {'

In [9]:
answer["text"]["content"]

'Hasta donde sé, el último turno que asignamos fue para Sylvia, tu madre, el 27 de agosto a las 10:00 AM con el Dr. Facundo Farias. Sin embargo, esta información fue proporcionada el 25 de agosto. Permíteme verificar si ha habido alguna actualización desde entonces. Por favor, dame un momento.'

Aca hay un error de razonamiento pero es mucho pedir para un prompt tan corto.

Posibles pasos siguientes:

- Navegar el grafo de relaciones del usuario y cambiar el punto focal actual. "este turno es para mi madre"
- Usar el mismo metodo para guardar cosas como "La madre del usuario tiene un turno para el dentista el 1 de enero de 2021"
- Sumar muchos mas datos personales y ver si hay confusiones
- Simular llamados a APIs desde diferentes relaciones del usuario y ver si hay confusiones


In [ ]:
user_input = "que edad tenia yo en 1988?"
filter = {"person_id": "20"}
db_results = db.similarity_search_with_score(query=user_input, embeddings=embeddings, filter=filter, k=2)
context_k2 = [f'\n {db_results[i][0].page_content} {db_results[i][0].metadata}\n---------------\n\n' for i in range(len(db_results))]
context = context_k2
for c in context:
    print(c)
system_prompt = f"""
You are the best personal assistant of SAIA Air.
Your job is to help users in their travel needs.
Use the user's name in your answers.
You will find past conversations between you and the user between backticks.
Use them to answer the user's question.
To reason about dates, have in mind that today is September 13th, 2023.
Wait before answering, check when the data was gathered first.
Take a deep breath and work on this problem step-by-step.



```{context}```

"""
answer = gpt_system_user(system_message=system_prompt, user_message=user_input, model="gpt-4")
answer["text"]["content"]


NameError: name 'db' is not defined